In [32]:
import numpy as np
import pandas as pd
import csv
import os
import glob
import tensorflow as tf

from PIL import Image

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.models import Model

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from keras.optimizers import Adam
from keras import backend as K

In [33]:
physical_devices = tf.config.list_physical_devices('GPU')
for i, device in enumerate(physical_devices):
    print(f"GPU {i}: {device}")


GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU 1: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [34]:
path = 'car_make_images/'
training_path = path + 'train'
testing_path = path + 'test'
validation_path = path + 'val'

In [35]:
training_data_generator = ImageDataGenerator(rescale = 1./255,
                              rotation_range = 359,
                              shear_range = 0.2,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              zoom_range = 0.2,
                              horizontal_flip = True,
                              vertical_flip = True,
                              preprocessing_function = None)

validation_data_generator = ImageDataGenerator(rescale = 1./255)
test_data_generator = ImageDataGenerator(rescale = 1./255)

In [36]:
size = 200
batch_size = 30  
num_classes = 39

training_generator = training_data_generator.flow_from_directory(training_path,
                                                                 target_size = (size, size),
                                                                 batch_size = 30,
                                                                 class_mode = "categorical",
                                                                 color_mode = 'grayscale',
                                                                 )

validation_generator = validation_data_generator.flow_from_directory(validation_path,
                                                                     target_size = (size, size),
                                                                     batch_size = 1,
                                                                     class_mode = "categorical",
                                                                     color_mode = 'grayscale',
                                                                     )

test_generator = test_data_generator.flow_from_directory(testing_path,
                                                         target_size = (size, size),
                                                         batch_size = 1,
                                                         class_mode = "categorical",
                                                         color_mode = 'grayscale',
                                                         )

Found 11573 images belonging to 39 classes.
Found 2813 images belonging to 39 classes.
Found 2871 images belonging to 39 classes.


In [37]:
with tf.device("/GPU:0"):
    model = Sequential()

    model.add(Input(shape=(size, size, 1)))

    # First Conv Block
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    # model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Second Conv Block
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    # model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Third Conv Block
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    # model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Fourth Conv Block
    model.add(Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    # model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Flatten and Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.15))
    model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(num_classes, activation='softmax', kernel_initializer='he_normal'))

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=0.001,
                decay_steps=10000,
                decay_rate=0.9)
    opt = Adam(learning_rate=lr_schedule)

In [39]:
def train_and_save(model, train_data, val_data, epochs, save_interval, model_save_path, history_save_path, custom_metrics=None, custom_optimizer=None):
    """
    Train a TensorFlow model and save it along with its history.
    """
    os.makedirs(model_save_path, exist_ok=True)

    if custom_optimizer:
        optimizer = custom_optimizer
    else:
        optimizer = 'adam'

    if custom_metrics:
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'] + custom_metrics)
    else:
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

    # Initialize variables
    initial_epoch = 0
    temp_history_data = []

    # Check if history file exists, if not create it
    if not os.path.exists(history_save_path):
        with open(history_save_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            columns = ['Epoch', 'Loss', 'Accuracy', 'Val_Loss', 'Val_Accuracy']
            if custom_metrics:
                for metric in custom_metrics:
                    metric_name = metric.__name__
                    columns.append(metric_name)
                    columns.append("Val_" + metric_name)
            csv_writer.writerow(columns)
    else:
        with open(history_save_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            last_row = None
            for row in csv_reader:
                last_row = row
            if last_row:
                initial_epoch = int(last_row[0])

    latest_model_file = max(glob.glob(f"{model_save_path}/model_e*.h5"), default=None, key=os.path.getctime)
    if latest_model_file is not None:
        print(f"Resuming from {latest_model_file}")
        model = tf.keras.models.load_model(latest_model_file, custom_objects={metric.__name__: metric for metric in custom_metrics})

    for epoch in range(initial_epoch + 1, epochs + initial_epoch + 1):
        print(f"Epoch {epoch}/{epochs + initial_epoch}")

        history = model.fit(train_data, validation_data=val_data)
        history_data = [epoch] + [history.history[key][0] for key in history.history]
        temp_history_data.append(history_data)

        if epoch % save_interval == 0 or epoch == epochs + initial_epoch:
            model_file_path = os.path.join(model_save_path, f"model_e{epoch}.h5")
            model.save(model_file_path)

            # Append to CSV at checkpoints
            with open(history_save_path, 'a', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                for row in temp_history_data:
                    csv_writer.writerow(row)

            # Clear temporary history data
            temp_history_data.clear()

            print(f"Saved model and history at epoch {epoch}")


In [40]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    actual_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (actual_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val


In [42]:
# Train the model and save it
with tf.device("/GPU:0"):
    train_and_save(
        model, 
        training_generator, 
        validation_generator, 
        epochs=40, 
        save_interval=10, 
        model_save_path="models", 
        history_save_path="history.csv", 
        custom_metrics=[f1_score],
        custom_optimizer=opt
    )


Resuming from models\model_e200.h5
Epoch 201/240
386/386 [==============================] - 89s 230ms/step - loss: 2.1695 - accuracy: 0.3930 - f1_score: 0.3481 - val_loss: 3.3219 - val_accuracy: 0.2606 - val_f1_score: 0.1674
Epoch 202/240
386/386 [==============================] - 90s 233ms/step - loss: 2.1732 - accuracy: 0.3926 - f1_score: 0.3383 - val_loss: 3.1785 - val_accuracy: 0.2755 - val_f1_score: 0.1802
Epoch 203/240
386/386 [==============================] - 92s 239ms/step - loss: 2.1516 - accuracy: 0.3971 - f1_score: 0.3501 - val_loss: 3.2532 - val_accuracy: 0.2659 - val_f1_score: 0.1724
Epoch 204/240
386/386 [==============================] - 106s 274ms/step - loss: 2.1419 - accuracy: 0.3989 - f1_score: 0.3492 - val_loss: 3.2700 - val_accuracy: 0.2627 - val_f1_score: 0.1770
Epoch 205/240
386/386 [==============================] - 104s 270ms/step - loss: 2.1526 - accuracy: 0.3966 - f1_score: 0.3442 - val_loss: 3.2990 - val_accuracy: 0.2695 - val_f1_score: 0.1809
Epoch 206/240